# 2.9 Integración de Fuentes

**Resumen Ejecutivo**: Combina datos heterogéneos (ventas, suscripciones) con `UNION ALL`, valida claves foráneas y prepara staging para ETL con controles de calidad.

---

## 🎯 Objetivos de Aprendizaje
- Unir fuentes con `UNION ALL` manteniendo granularidad consistente.
- Detectar claves huérfanas (productos sin ventas, clientes sin actividad).
- Crear vistas consolidadas para reporting multi-fuente.

## 🧰 Prerrequisitos
- Haber completado `08_control_version_datos.ipynb`.
- Conocer `UNION` vs. `UNION ALL` y validación de claves con `LEFT JOIN`.
- Familiaridad con staging y ETL conceptual.

---

## 📚 Conceptos Clave
- **UNION ALL**: combina resultados sin eliminar duplicados (más rápido).
- **Granularidad consistente**: alinear fechas, métricas, dimensiones antes de unir.
- **Validación de claves**: `LEFT JOIN` inverso detecta registros huérfanos.
- **Vista consolidada**: abstrae complejidad de múltiples fuentes para consumidores.

> ⚠️ Unir granularidades incompatibles genera métricas incorrectas; `UNION` sin `ALL` puede perder duplicados válidos.

Estrategias: staging temporal, validación de claves, mapeo de dominios (segmento, región).

In [ ]:
-- Unificar ventas + suscripciones (métrica de ingresos combinada conceptual)
SELECT fecha, 'VENTA' AS tipo, SUM(cantidad*p.precio_unitario) AS ingresos
FROM dbo.fact_ventas fv
JOIN dbo.dim_productos p ON fv.producto_id = p.producto_id
GROUP BY fecha
UNION ALL
SELECT fecha_inicio AS fecha, 'SUSCRIPCION' AS tipo, SUM(mrr) AS ingresos
FROM dbo.fact_suscripciones
GROUP BY fecha_inicio;

🟢 Ejercicio: Agregar columna derivada origen ('ventas'/'suscripciones').

🟠 Ejercicio: Detectar claves de producto inexistentes en tabla de ventas (LEFT JOIN inverso).

In [ ]:
SELECT p.producto_id, p.nombre
FROM dbo.dim_productos p
LEFT JOIN dbo.fact_ventas fv ON p.producto_id = fv.producto_id
WHERE fv.producto_id IS NULL;

🔴 Reto: Crear vista consolidada de ingresos diarios sumando ventas y suscripciones y calcular participación porcentual de cada tipo.

Errores comunes: unir granularidades incompatibles; usar UNION en vez de UNION ALL perdiendo duplicados necesarios; omitir validación de claves foráneas.

# Créditos

Este material fue revisado y enriquecido parcialmente mediante asistencia de IA (OpenAI y Claude); la validación y decisiones editoriales finales son humanas.

---
## Navegación
[⬅️ Anterior](08_control_version_datos.ipynb) | [Siguiente ➡️](10_proyecto_intermedio.ipynb)
---
